# Harmonic Oscillator
## Comparison of Numerical Integration Methods

### Usman Alim
### CPSC 587/687: Winter 2020


In [22]:
import ipywidgets as widgets
from ipywidgets import interact
import plotly.graph_objs as go

import numpy as np

In [23]:
# Parameters that will stay fixed. For this demo, this includes
# the initial state, the amplitude, and the time interval.
# We are assuming that the particle has unit mass.

# amplitude
A = 1

# Duration
T = 10.0

# Particle starts with amplitude A at rest.
s0 = np.array([A, 0]) 

In [41]:
# Function that returns the analytic solution
# Input time is assumed to be a numpy array
def analyticSolution(k, d, t):
    w = np.sqrt(k)
    z = d / (2*np.sqrt(k))
    return A * np.exp(-z*w*t) * np.cos(w*np.sqrt(1-z*z)*t)

## Various integrators
### Each definition below implements one time-step

In [47]:
# One step of forward Euler
def forward_euler_step(h=0.1, k=1, d=0):
    s = s0
    M = np.array([[1, h],[-h*k, 1-d*h]])
    while True:
        yield s
        s = M.dot(s)
        
# One step of semi-implicit Euler
def si_euler_step(h=0.1, k=1, d=0):
    s = s0
    M = np.array([[1-k*h*h, h*(1-d*h)],[-k*h, 1-d*h]])
    while True:
        yield s
        s = M.dot(s)    
        
# One step of RK2
def RK2_step(h=0.1, k=1, d=0):
    s = s0
    M = np.array([[0, 1],[-k, -d]])
    while True:
        yield s
        k1 = M.dot(s)
        k2 = M.dot(s + 0.5*h*k1)
        s = s + h*k2

# One step of RK4
def RK4_step(h=0.1, k=1, d=0):
    s=s0
    M = np.array([[0, 1],[-k, -d]])
    while True:
        yield s
        k1 = M.dot(s)
        k2 = M.dot(s + 0.5*h*k1)
        k3 = M.dot(s + 0.5*h*k2)
        k4 = M.dot(s+ h*k3)
        s = s + (k1 + 2*k2 + 2*k3 + k4)*h/6 

In [28]:
# Compute the solution using a specified method
def nsol(h,k,d, method):
    feg = method(h, k, d)
    N = int(T / h)
    return np.array([next(feg) for i in range(N)])

## Widget to Compare Forward Euler and Semi-Implicit Euler

In [57]:
#Starting parameters
h = 0.1
k = 2
d = 0

t = np.linspace(0,T,int(T/h))
scatt_truth_ND = go.Scatter(x=t, y=analyticSolution(k,0,t), 
                  line_color='rgba(0, 0, 0, 1)', name='Truth (no damping)')
scatt_truth_D = go.Scatter(x=t, y=analyticSolution(k,d,t), 
                  line_color='rgba(100, 100, 100, 1)', name='Truth (with damping)')

scatt_FE = go.Scatter(x=t, y=nsol(h, k, d, forward_euler_step)[:,0], 
                      line_color='rgba(31,120,180,0.9)',
                      mode='lines+markers', marker_size=4,
                      name='Forward Euler')
scatt_SIE = go.Scatter(x=t, y=nsol(h, k, d, si_euler_step)[:,0], 
                      line_color='rgba(51,160,144,0.9)',
                      mode='lines+markers', marker_size=4,
                      name='Semi-implicit Euler')



# Note the use of FigureWidget rather than Figure
figE = go.FigureWidget()
figE.add_trace(scatt_truth_ND)
figE.add_trace(scatt_truth_D)
figE.add_trace(scatt_FE)
figE.add_trace(scatt_SIE)


figE.update_layout(
    title="Euler vs. Semi-implicit Euler",
    xaxis_title="t",
    yaxis_title="x"
)


@interact(h=widgets.BoundedFloatText(
    value=h,description='step:',disabled=False,min=0.001,max=1,step=0.01),
          k=widgets.BoundedFloatText(
              value=k,description='k:',disabled=False, min=1, max=10, 
              step=1),
          d=widgets.BoundedFloatText(
    value=d,description='d:',disabled=False,min=0,max=10,step=0.1)
         )
def updateFE(h=h, k=k, d=d):
    figE.update_traces(x=t,y=analyticSolution(k,0,t),
                       selector=dict(type="scatter", name='Truth (no damping)'))
    figE.update_traces(x=t,y=analyticSolution(k,d,t),
                       selector=dict(type="scatter", name='Truth (with damping)'))
    figE.update_traces(x=np.linspace(0,T,int(T/h)),
                        y=nsol(h, k, d, forward_euler_step)[:,0],
      selector=dict(type="scatter", name="Forward Euler"))
    figE.update_traces(x=np.linspace(0,T,int(T/h)),
                        y=nsol(h, k, d, si_euler_step)[:,0],
      selector=dict(type="scatter", name="Semi-implicit Euler"))
    figE.update_yaxes(range=[-2, 2])
                       
figE

interactive(children=(BoundedFloatText(value=0.1, description='step:', max=1.0, min=0.001, step=0.01), Bounded…

FigureWidget({
    'data': [{'line': {'color': 'rgba(0, 0, 0, 1)'},
              'name': 'Truth (no damping)'…

## Widget to Compare RK2 and RK4

In [58]:
#Starting parameters
h = 0.1
k = 2
d = 0

t = np.linspace(0,T,int(T/h))
scatt_truth = go.Scatter(x=t, y=analyticSolution(k,d,t), 
                  line_color='rgba(0, 0, 0, 1)', name='Truth')

scatt_RK2 = go.Scatter(x=t, y=nsol(h, k, d, RK2_step)[:,0], 
                      line_color='rgba(217,95,2,0.9)',
                      mode='lines+markers', marker_size=4,
                      name='RK2')
scatt_RK4 = go.Scatter(x=t, y=nsol(h, k, d, RK4_step)[:,0], 
                      line_color='rgba(27,158,119,0.9)',
                      mode='lines+markers', marker_size=4,
                      name='RK4')

# Note the use of FigureWidget rather than Figure
figRK = go.FigureWidget()
figRK.add_trace(scatt_truth)
figRK.add_trace(scatt_RK2)
figRK.add_trace(scatt_RK4)

figRK.update_layout(
    title="RK2 vs. RK4",
    xaxis_title="t",
    yaxis_title="x"
)


@interact(h=widgets.BoundedFloatText(
    value=h,description='step:',disabled=False,min=0.001,max=1,step=0.01),
          k=widgets.BoundedFloatText(
              value=k,description='k:',disabled=False, min=1, max=10, 
              step=1),
          d=widgets.BoundedFloatText(
    value=d,description='d:',disabled=False,min=0,max=10,step=0.1)
         )
def updateRK(h=h, k=k, d=d):
    figRK.update_traces(x=t,y=analyticSolution(k,d,t),
                       selector=dict(type="scatter", name='Truth'))
    figRK.update_traces(x=np.linspace(0,T,int(T/h)),
                        y=nsol(h, k, d, RK2_step)[:,0],
      selector=dict(type="scatter", name="RK2"))
    figRK.update_traces(x=np.linspace(0,T,int(T/h)),
                        y=nsol(h, k, d, RK4_step)[:,0],
      selector=dict(type="scatter", name="RK4"))
    figRK.update_yaxes(range=[-2, 2])
                       
figRK

interactive(children=(BoundedFloatText(value=0.1, description='step:', max=1.0, min=0.001, step=0.01), Bounded…

FigureWidget({
    'data': [{'line': {'color': 'rgba(0, 0, 0, 1)'},
              'name': 'Truth',
           …